In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

# 3) Relaciones entre eventos

## 3.1)Relación viewed product y checkout. 

In [441]:
coincidencias = 0 
personas = tf.loc[:,['event','person','sku']]
#personas por el evento checkout
personas_conv = personas[(personas['event'] == 'conversion')]
#personas por el evento viewed
personas_viewed = personas[(personas['event'] == 'viewed product')].drop_duplicates()

interseccion_de_personas = pd.merge(personas_conv, personas_viewed, on=['person','sku'], how='inner')
#CANTIDAD DE  COMPRAS POR PERSONAS QUE COMPRARON EL PRODUCTO QUE MIRARON VARIAS VECES.
cantidad_de_compras_iguales = union_de_personas.groupby('person').size().to_frame('cantidad de compras').groupby('cantidad de compras').size().to_frame('cantidad de personas')
cantidad_de_compras_iguales

,cantidad de personas
cantidad de compras,
1,372
2,103
3,37
4,19
5,6
6,8
7,5
8,3
10,1


**3.1.1)¿Cuántas personas compraron el mismo producto que vieron?**

In [466]:
coincidencias = 0 
#personas = tf.loc[:,['event','person','sku']]
#personas por el evento checkout
personas_conv = personas[(personas['event'] == 'conversion')]
#personas por el evento viewed
personas_viewed = personas[(personas['event'] == 'viewed product')].drop_duplicates()#
#hago un drop por si alguien vio muchas veces el producto antes de comprarlo
#si una persona vio muchas veces el producto antes de comprarlo

interseccion_de_personas = pd.merge(personas_conv, personas_viewed, on=['person','sku'], how='inner')
tf.event.value_counts()

viewed product       528931
brand listing         98635
visited site          87378
ad campaign hit       82827
generic listing       67534
searched products     56073
search engine hit     50957
checkout              33735
staticpage             3598
conversion             1172
lead                    448
Name: event, dtype: int64

** 3.1.2)¿Cuántas personas no compraron el producto que vieron?**

In [459]:
interseccion_de_personas_2 = pd.merge(personas_conv, personas_viewed, on = ['person'], how = 'inner',copy = False)
interseccion_de_personas_2['compraron el mismo'] = (interseccion_de_personas_2['sku_x'] == interseccion_de_personas_2['sku_y'])
interseccion_de_personas_2.drop_duplicates()
interseccion_de_personas_2['compraron el mismo'].value_counts()

False    51861
True       935
Name: compraron el mismo, dtype: int64